In [ ]:
import math 
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt 

df = pd.read_csv('../input/tcs-dataset-from-2015-to-2021/TCS.NSE.csv')
df

In [ ]:
df1 = df['Close']
df1.shape

In [ ]:
x = df1.isnull()
for i in range(0,1666):
  if x[i]:
    df.iloc[i] = df.iloc[i+1]
df1 = pd.DataFrame(df1)

In [ ]:
test_set = df['Close'].iloc[1332:1666]


In [ ]:
plt.figure(figsize=(16,8))
plt.plot(df['Close'], 'black')
plt.title('Closing price of TCS')
plt.xlabel('Date', fontsize = 18)
plt.ylabel('Rupees', fontsize = 18)

In [ ]:
scaler=MinMaxScaler(feature_range=(-1,1))
df2 = scaler.fit_transform(np.array(df1).reshape(-1,1))

df2.shape

In [ ]:
training_size=int(len(df2)*0.80)
test_size=len(df2)-training_size
test_data = df2[training_size:len(df2)]
train_data = df2[0:training_size]
train_data.shape

In [ ]:
import numpy
def create_dataset(dataset, time_step=10):
    dataX, dataY = [],[]
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [ ]:
time_step = 5
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [ ]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test =X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
model = keras.Sequential()

model.add(LSTM(50,return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(loss='mean_squared_error',optimizer=(tf.keras.optimizers.Adam(lr = 0.01)))


In [ ]:
history = model.fit(X_train,y_train,epochs=100)

In [ ]:
plt.plot(history.history['loss'])


In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
test_range = np.arange(1332,1660)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(df['Close'], 'black')
plt.plot(train_predict, 'red')
plt.plot( test_range, test_predict, 'g')
plt.title('Closing price of TCS')
plt.xlabel('Date', fontsize = 18)
plt.ylabel('Rupees', fontsize = 18)